# 第一章 数据集结构和算法(2)

### 1.5 实现一个优先级队列 

**问：怎样实现一个按优先级排序的队列？并且在这个队列上面每次 pop 操作总是返回
优先级最高的那个元素**

In [11]:
import heapq


class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
            
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item)) 
        self._index += 1
        
    def pop(self):
        return heapq.heappop(self._queue)[-1]

In [12]:
q = PriorityQueue() 
q.push('foo', 1)
q.push('bar', 5)
q.push('spam', 4)
q.push('grok', 1)
q.pop()

'bar'

In [13]:
q.pop()

'spam'

In [14]:
q.pop()

'foo'

In [15]:
q.pop()

'grok'

**仔细观察可以发现，第一个 pop() 操作返回优先级最高的元素。另外注意到如果
两个有着相同优先级的元素 ( foo 和 grok )，pop 操作按照它们被插入到队列的顺序返
回的**

这 一 小 节 我 们 主 要 关 注 heapq 模 块 的 使 用。 函 数 heapq.heappush() 和
heapq.heappop() 分别在队列 queue 上插入和删除第一个元素，并且队列 queue 保证
第一个元素拥有最小优先级 (1.4 节已经讨论过这个问题)。 heappop() 函数总是返回”
最小的” 的元素，这就是保证队列 pop 操作返回正确元素的关键。另外，由于 push 和
pop 操作时间复杂度为 O(log N)，其中 N 是堆的大小，因此就算是 N 很大的时候它们
运行速度也依旧很快

在上面代码中，队列包含了一个 (-priority, index, item) 的元组。优先级为负
数的目的是使得元素按照优先级从高到低排序。这个跟普通的按优先级从低到高排序
的堆排序恰巧相反。

index 变量的作用是保证同等优先级元素的正确排序。通过保存一个不断增加的
index 下标变量，可以确保元素按照它们插入的顺序排序。而且， index 变量也在相
同优先级元素比较的时候起到重要作用

**如果你想在多个线程中使用同一个队列，那么你需要增加适当的锁和信号量机制**

### 1.6 字典中的键映射多个值

**问：怎样实现一个键对应多个值的字典 (也叫 multidict )？
**

- 一个字典就是一个键对应一个单值的映射。如果你想要一个键映射多个值，那么你
就需要将这多个值放到另外的容器中，比如列表或者集合里面。比如，你可以像下面
这样构造这样的字典：

In [16]:
d = {
    'a' : [1, 2, 3],
    'b' : [4, 5]
    }

e = {
    'a' : {1, 2, 3},
    'b' : {4, 5}
    }


- 使用 collections 模块中的 defaultdict 来构造这样的字典。
defaultdict 的一个特征是它会自动初始化每个 key 刚开始对应的值，所以你只需要
关注添加元素操作了。比如：

In [17]:
from collections import defaultdict 


d = defaultdict(list) 
d['a'].append(1)
d['a'].append(2)
d['b'].append(22)
d

defaultdict(list, {'a': [1, 2], 'b': [22]})

In [18]:
d = defaultdict(set) 
d['a'].add(1)
d

defaultdict(set, {'a': {1}})

In [19]:
# 需要注意的是， defaultdict 会自动为将要访问的键 (就算目前字典中并不存在这
# 样的键) 创建映射实体。如果你并不需要这样的特性，你可以在一个普通的字典上使用
# setdefault() 方法来代替。比如：

d = {} 
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)
d

{'a': [1, 2], 'b': [4]}

### 1.7 字典排序

**问：你想创建一个字典，并且在迭代或序列化这个字典的时候能够控制元素的顺序。**

*为 了 能 控 制 一 个 字 典 中 元 素 的 顺 序， 你 可 以 使 用 collections 模 块 中 的OrderedDict 类(根据输入顺序排序)。*

In [2]:
from collections import OrderedDict 


d = OrderedDict()
d['foo'] = 1 
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4 
for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


当你想要构建一个将来需要序列化或编码成其他格式的映射的时候， OrderedDict
是非常有用的

In [3]:
# 比如，你想精确控制以 JSON 编码后字段的顺序，你可以先使用OrderedDict 来构建这样的数据：

import json 
json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。每次当一个新的元
素插入进来的时候，它会被放到链表的尾部。对于一个已经存在的键的重复赋值不会
改变键的顺序。

需要注意的是，一个 OrderedDict 的大小是一个普通字典的两倍，因为它内部维
护着另外一个链表。所以如果你要构建一个需要大量 OrderedDict 实例的数据结构的
时候 (比如读取 100,000 行 CSV 数据到一个 OrderedDict 列表中去)，那么你就得仔细
权衡一下是否使用 OrderedDict 带来的好处要大过额外内存消耗的影响。


### 1.8 字典的运算 

**问：怎样在数据字典中执行一些计算操作 (比如求最小值、最大值、排序等等)？**

In [4]:
# 考虑下面的股票名和价格映射字典：

prices = {
'ACME': 45.23,
'AAPL': 612.78,
'IBM': 205.55,
'HPQ': 37.20,
'FB': 10.75
}

In [5]:
# 为了对字典值执行计算操作，通常需要使用 zip() 函数先将键和值反转过来。
# 比如，下面是查找最小和最大股票价格和股票值的代码： 

min_price = min(zip(prices.values(), prices.keys()))
max_price = max(zip(prices.values(), prices.keys()))
print(min_price) 
print(max_price)

(10.75, 'FB')
(612.78, 'AAPL')


In [6]:
# 可以使用 zip() 和 sorted() 函数来排列字典数据：

prices_sorted = sorted(zip(prices.values(), prices.keys()))
prices_sorted

[(10.75, 'FB'),
 (37.2, 'HPQ'),
 (45.23, 'ACME'),
 (205.55, 'IBM'),
 (612.78, 'AAPL')]

**执行这些计算的时候，需要注意的是 zip() 函数创建的是一个只能访问一次的迭
代器。比如，下面的代码就会产生错误**

In [8]:
prices_and_names = zip(prices.values(), prices.keys())
print('min:', min(prices_and_names))
print('max', max(prices_and_names))

min: (10.75, 'FB')


ValueError: max() arg is an empty sequence

如果你在一个字典上执行普通的数学运算，你会发现它们仅仅作用于键，而不是
值。比如：

In [9]:
min(prices)

'AAPL'

尝试着使用字典的 values() 方法来解决这个问题,但不会返回键值

In [10]:
min(prices.values()) 

10.75

可以在 min() 和 max() 函数中提供 key 函数参数来获取最小值或最大值对应的
键的信息。

In [11]:
min(prices, key=lambda k: prices[k])

'FB'

In [13]:
min_value = prices[min(prices, key=lambda k: prices[k])]
min_value

10.75

### 1.9 查找两字典的相同点

**问：怎样在两个字典中寻寻找相同点 (比如相同的键、相同的值等等)？**

In [14]:
a = {
    'x' : 1,
    'y' : 2,
    'z' : 3
    }
b = {
    'w' : 10,
    'x' : 11,
    'y' : 2
    }

In [15]:
# 为了寻找两个字典的相同点，可以简单的在两字典的 keys() 或者 items() 方法返回结果上执行集合操作

print(a.keys() & b.keys())
print(a.keys() - b.keys())
print(a.items() & b.items())

{'y', 'x'}
{'z'}
{('y', 2)}


这些操作也可以用于修改或者过滤字典元素。比如，假如你想以现有字典构造一个
排除几个指定键的新字典。下面利用字典推导来实现这样的需求：


In [17]:
d = {key: a[key] for key in a.keys() - {'z', 'w'}}
d

{'x': 1, 'y': 2}

### 1.10 删除序列相同元素并保持顺序

**问:怎样在一个序列上面保持元素顺序的同时消除重复的值？**

如果序列上的值都是 hashable 类型(可哈希)，那么可以很简单的利用集合或者生成器来解
决这个问题。比如：

In [18]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

a = [1, 5, 2, 1, 9, 1, 5, 10] 
list(dedupe(a))

[1, 5, 2, 9, 10]

如果你想消除元素不可哈
希 (比如 dict 类型) 的序列中重复元素的话，你需要将上述代码稍微改变一下，就像
这样

In [24]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)
            
# 这里的 key 参数指定了一个函数，将序列元素转换成 hashable 类型 

a = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]
list(dedupe(a, key=lambda d: (d['x'],d['y'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

如果你仅仅就是想消除重复元素，通常可以简单的构造一个集合,但不会维持顺序

In [25]:
a = [1, 3, 5, 1, 5, 3, 4]
set(a)

{1, 3, 4, 5}

### 1.11 命名切片

**问：你的程序已经出现一大堆已无法直视的硬编码切片下标，然后你想清理下代码。**

假定你有一段代码要从一个记录字符串中几个固定位置提取出特定的数据字段 (比
如文件或类似格式)

In [27]:
###### 0123456789012345678901234567890123456789012345678901234567890'
record = '....................100 .......513.25 ..........'
cost = int(record[20:23]) * float(record[31:37])
cost

51325.0

与其那样写，为什么不想这样命名切片呢：

In [29]:
SHARES = slice(20, 23)
PRICE = slice(31, 37)
cost = int(record[SHARES]) * float(record[PRICE])
cost

51325.0

内置的 slice() 函数创建了一个切片对象，可以被用在任何切片允许使用的地方。

In [30]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2, 4)
print(items[2: 4]) 
print(items[a])

[2, 3]
[2, 3]


In [31]:
items[a] = [23, 34]
items

[0, 1, 23, 34, 4, 5, 6]

如果你有一个切片对象 a，你可以分别调用它的 a.start , a.stop , a.step 属性来
获取更多的信息

In [32]:
a = slice(5, 50, 2)
print(a.start)
print(a.stop)
print(a.step)

5
50
2


另外，你还能通过调用切片的 indices(size) 方法将它映射到一个确定大小的序
列上，这个方法返回一个三元组 (start, stop, step) ，所有值都会被合适的缩小以
满足边界限制，从而使用的时候避免出现 IndexError 异常。

In [34]:
s = 'HelloWorld'
a.indices(len(s))
for i in range(*a.indices(len(s))):
    print(s[i])

W
r
d
